In [31]:
import numpy as np
from qiskit import QuantumCircuit, Aer, execute
from qiskit.visualization import plot_histogram

# Define the list of numbers
numbers = [1, 2, -3, 5, -4]

In [32]:
# Determine the number of qubits needed to represent the numbers
max_num = max(map(abs, numbers))
num_qubits = max(2, max_num.bit_length() + 2)

In [33]:
qc = QuantumCircuit(num_qubits, num_qubits)
qc.draw()

q_0: 
     
q_1: 
     
q_2: 
     
q_3: 
     
q_4: 
     
c: 5/

In [34]:
for qubit in range(num_qubits):
    qc.h(qubit)
    
qc.draw()

┌───┐
q_0: ┤ H ├
     ├───┤
q_1: ┤ H ├
     ├───┤
q_2: ┤ H ├
     ├───┤
q_3: ┤ H ├
     ├───┤
q_4: ┤ H ├
     └───┘
c: 5/═════

In [35]:
#oracle circuit
oracle = QuantumCircuit(num_qubits,name='oracle')
oracle.cz(num_qubits-2,num_qubits-1)
oracle.to_gate()
oracle.draw()

q_0: ───
        
q_1: ───
        
q_2: ───
        
q_3: ─■─
      │ 
q_4: ─■─

In [36]:
backend = Aer.get_backend('statevector_simulator')
qc.append(oracle,list(range(num_qubits)))
qc.draw()

┌───┐┌─────────┐
q_0: ┤ H ├┤0        ├
     ├───┤│         │
q_1: ┤ H ├┤1        ├
     ├───┤│         │
q_2: ┤ H ├┤2 oracle ├
     ├───┤│         │
q_3: ┤ H ├┤3        ├
     ├───┤│         │
q_4: ┤ H ├┤4        ├
     └───┘└─────────┘
c: 5/════════════════

In [37]:
job = execute(qc,backend)
result = job.result()
sv = result.get_statevector()
# np.around(sv,2)

In [38]:
diff = QuantumCircuit(num_qubits,name='diffusion')
for qubit in range(num_qubits):
    diff.h(qubit)
    diff.x(qubit)

for qubit in range(num_qubits - 1):
    diff.cz(qubit, num_qubits - 1)

for qubit in range(num_qubits):
    diff.x(qubit)
    diff.h(qubit)

diff.x(num_qubits-1)
    
diff.to_gate()
diff.draw()

┌───┐┌───┐   ┌───┐┌───┐                    
q_0: ┤ H ├┤ X ├─■─┤ X ├┤ H ├────────────────────
     ├───┤├───┤ │ └───┘├───┤┌───┐               
q_1: ┤ H ├┤ X ├─┼───■──┤ X ├┤ H ├───────────────
     ├───┤├───┤ │   │  └───┘├───┤┌───┐          
q_2: ┤ H ├┤ X ├─┼───┼────■──┤ X ├┤ H ├──────────
     ├───┤├───┤ │   │    │  └───┘├───┤┌───┐     
q_3: ┤ H ├┤ X ├─┼───┼────┼────■──┤ X ├┤ H ├─────
     ├───┤├───┤ │   │    │    │  ├───┤├───┤┌───┐
q_4: ┤ H ├┤ X ├─■───■────■────■──┤ X ├┤ H ├┤ X ├
     └───┘└───┘                  └───┘└───┘└───┘

In [39]:
qc.append(diff,list(range(num_qubits)))
qc.draw()

┌───┐┌─────────┐┌────────────┐
q_0: ┤ H ├┤0        ├┤0           ├
     ├───┤│         ││            │
q_1: ┤ H ├┤1        ├┤1           ├
     ├───┤│         ││            │
q_2: ┤ H ├┤2 oracle ├┤2 diffusion ├
     ├───┤│         ││            │
q_3: ┤ H ├┤3        ├┤3           ├
     ├───┤│         ││            │
q_4: ┤ H ├┤4        ├┤4           ├
     └───┘└─────────┘└────────────┘
c: 5/══════════════════════════════

In [40]:
# Measure the qubits
qc.measure(range(num_qubits), range(num_qubits))
qc.draw()

┌───┐┌─────────┐┌────────────┐┌─┐            
q_0: ┤ H ├┤0        ├┤0           ├┤M├────────────
     ├───┤│         ││            │└╥┘┌─┐         
q_1: ┤ H ├┤1        ├┤1           ├─╫─┤M├─────────
     ├───┤│         ││            │ ║ └╥┘┌─┐      
q_2: ┤ H ├┤2 oracle ├┤2 diffusion ├─╫──╫─┤M├──────
     ├───┤│         ││            │ ║  ║ └╥┘┌─┐   
q_3: ┤ H ├┤3        ├┤3           ├─╫──╫──╫─┤M├───
     ├───┤│         ││            │ ║  ║  ║ └╥┘┌─┐
q_4: ┤ H ├┤4        ├┤4           ├─╫──╫──╫──╫─┤M├
     └───┘└─────────┘└────────────┘ ║  ║  ║  ║ └╥┘
c: 5/═══════════════════════════════╩══╩══╩══╩══╩═
                                    0  1  2  3  4

In [41]:
simulator = Aer.get_backend('qasm_simulator')
job = execute(qc, simulator, shots=1024)

# Get and plot the measurement results
result = job.result()
counts = result.get_counts(qc)
# plot_histogram(counts)

In [42]:
# print(counts)

In [43]:
def decimal_to_signed_twos_complement(decimal_num, bit_width):
    if decimal_num < 0:
        # Calculate the absolute value of the decimal number
        abs_decimal = abs(decimal_num)
        
        # Convert the absolute value to binary
        binary_repr = bin(abs_decimal)[2:].zfill(bit_width)
        
        # Apply 2's complement by inverting the bits and adding 1
        inverted_bits = ''.join(['1' if bit == '0' else '0' for bit in binary_repr])
        inverted_bits = inverted_bits[-bit_width:]  # Ensure the same number of bits
        twos_complement = bin(int(inverted_bits, 2) + 1)[2:]
        
        return twos_complement
    else:
        # Convert the positive decimal number to binary
        binary_repr = bin(decimal_num)[2:].zfill(bit_width)
        
        return binary_repr


In [44]:
for num in numbers:
    if num < 0:
        tar = num
        break

In [45]:
check = decimal_to_signed_twos_complement(tar, num_qubits)

In [46]:
# Check if a negative number was found
if check in counts:
    print(True)
else:
    print(False)

True
